# A good tourist city

## Introduction/Business Problem

Tourism is an important economic income for cities. Some cities are attractive because of their beaches, their building, their history and much more. But there are also some cities which wouldn't be nice to go for a vacation. In this project I will train in a machine learning model that can determine how attractive a given city would be for tourists based on other attractive cities.

The problem this project will solve is knowing which city is a good place to visit. It can also help local/city governments make their cities more attractive based on the results of the algorithm.

## Data

The data used for this project will be obtained from Foursquare using the [Foursquare API](https://developer.foursquare.com/docs/places-api/). Only relevant venue categories will be requested, such us:

- Outdoors & Recreation
- Shop & Service
- Travel & Transport
    - Hotel
    - Taxi
    - Tourist Information Center
- Etc.

The various Foursquare venue categories are listed in [this link](https://developer.foursquare.com/docs/build-with-foursquare/categories/).

In addition, the cities categorized as "good tourist cities" will be chosen from articles like [this one](https://www.planetware.com/world/top-cities-in-the-world-to-visit-eng-1-39.htm). Venues data obtained from these cities will be fed into the machine learning algorithm to create the model.